# Emotion Detection Unsing OpenCV and Keras
In these project we will use keras:the Python deep learning api for emotion detection using live camera of your system.
### Importing required modules

In [1]:
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

### Defining constant values

* num_classses : The number of classes or the emotions that will be dealing with in the training our model                
* img_rows,img_cols : These number define the size of the image array that we will be feeding the neural network        
* batch_size :  This variable defines the batch size.The batch size is a number of samples processed before the model is updated. The number of epochs is the number of complete passes through the training dataset. The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.

In [2]:
num_classes=7
img_rows,img_cols=48,48
batch_size=32

### Upload the data direction

In [3]:
train_data_dir="/kaggle/input/emotion-detection-fer/train"
test_data_dir="/kaggle/input/emotion-detection-fer/test"

### Image augmentation
Image Data Augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.

The train_datagen variable will artificially expand the dataset using the following:

* rotation_range: Degree range for random rotations. Here i am using 30 degrees.
* shear_range: Shear Intensity (Shear angle in counter-clockwise direction in degrees). Here i am using 0.3 as shear range.
* zoom_range: Range for random zoom.Here i am using 0.3 as zoom range.
* width_shift_range: This shifts the images by a value across its width.
* height_shift_range : This shifts the images by a value across its height.
* horizontal_flip: This flips the images horizontally.
* fill_mode: This is used to fill in the pixels after making changes to the orientation of the images by the above used methods. Here i am using ‘nearest’ as the fill mode as i am instructing it to fill the missing pixels in the image with the nearby pixels.
Here I am just resclaing the validation data and not performing any other augmentaions as i want to check the model with raw data that is different from the data used in the training of the model.

In [4]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [5]:
 train_generator = train_datagen.flow_from_directory(
 train_data_dir, color_mode='grayscale', target_size=(img_rows,img_cols),
 batch_size=32, class_mode='categorical', shuffle=True)
 test_generator = test_datagen.flow_from_directory(
 test_data_dir, color_mode='grayscale', target_size=(img_rows,img_cols),
 batch_size=batch_size, class_mode='categorical',shuffle=True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


The options given are:

* directory: The directory of the dataset.
* color_mode: Here i am converting the images to gray-scale as i am not interested in the color of the images but only the expressions.
* target_size: Convert the images to a uniform size.
* batch_size: To make baches of data to train.
* class_mode: Here i am using ‘categorical’ as the class mode as i am categorizing my images into 5 classes.
* shuffle: To shuffle the dataset for better training.

### Build the model
Here we are using a Sequential model which defines that all the layers in the network will be one after the other sequentially and storing it in a variable model.

In [6]:
model=Sequential()

2022-11-28 12:07:57.059982: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


The network consists of 7 blocks

Before compiling we will create 3 things using **keras.callbacks**: 
**Block-1 layers in the order of occurrence are as follows :
* Conv2D layer- This layer creates a convolutional layer for the network. Here i am creating a layer with 32 filters and a filter size of (3,3) with padding=’same’ to pad the image and using the kernel initializer he_normal. I have added 2 convolutional layers each followed by an activation and batch normalization layers.
* Activation layer — I am using a elu activation.
* BatchNormalization — Normalize the activations of the previous layer at each batch, i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.
* MaxPooling2D layer — Downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis.Here i have used the pool_size as (2,2).
* Dropout: Dropout is a technique where randomly selected neurons are ignored during training. Here i am using dropout as 0.5 which means that it will ignore half of the neurons.
**Block-2 layers in the order of occurrence are as follows :
* Same layers as block-1 but the convolutional layers have 64 filters.
**Block-3 layers in the order of occurrence are as follows :
* Same layers as block-1 but the convolutional layers have 128 filters.
**Block-4 layers in the order of occurrence are as follows :
* Same layers as block-1 but the convolutional layers have 256 filters.
**Block-5 layers in the order of occurrence are as follows :
* Flatten layer — To flatten the output of the previous layers in a flat layer or in other words in the form of a vector.
* Dense layer — A densely connected layer where each neuron is connected to every other neuron. Here i am using 64 units or 64 neurons with a kernel initializer — he_normal.
* These layers are followed by activation layer with elu activation , batch normalization and finally a dropout with 50% dropout.
**Block-6 layers in the order of occurrence are as follows :
* Same layers as block 5 but without flatten layer as the input for this block is already flattened.
**Block-7 layers in the order of occurrence are as follows :
* Dense layer — Finally in the final block of the network i am using num_classes to create a dense layer having units=number of classes with a he_normal initializer.
* Activation layer — Here i am using a softmax layer which is used for multi-class classifications.

In [7]:
#first block
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

### the overall structure of the model:

In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

### Compile and train 

In [9]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Checkpoint( Function — ModelCheckpoint() )
It will monitor the validation loss and will try to minimize the loss using the mode=’min’ property. When the checkpoint is reached it will save the best trained weights. Verbose=1 is just for visualization when the code created checkpoint.Here i am using it’s following parameters:

file-path: Path to save the model file.Here i am saving the model file with the name EmotionDetectionModel.h5
monitor: Quantity to monitor.Here i am monitoring the validation loss.
mode: One of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity.
save_best_only: If save_best_only=True, the latest best model according to the quantity monitored will not be overwritten.
verbose: int. 0: quiet, 1: update messages.
Early Stopping ( Function — EarlyStopping() )
This will stop the execution early by checking the following properties.

monitor: Quantity to monitor.Here i am monitoring the validation loss.
min_delta: Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.Here i have given it 0.
patience: Number of epochs with no improvement after which training will be stopped. Here i have given it 3.
restore_best_weights: Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.Here i have given it True.
verbose: int. 0: quiet, 1: update messages.
Reduce Learning Rate ( Function — ReduceLROnPlateau() )
Models often benefit from reducing the learning rate by a factor of 2–10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a ‘patience’ number of epochs, the learning rate is reduced. I have used the following properties for this.

monitor: To monitor a particular loss. Here i am monitoring the validation loss.
factor: Factor by which the learning rate will be reduced. new_lr = lr * factor. Here i am using 0.2 as factor.
patience: Number of epochs with no improvement after which learning rate will be reduced.Here i am using 3.
min_delta: Threshold for measuring the new optimum, to only focus on significant changes.
verbose: int. 0: quiet, 1: update messages.

In [10]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

model.compile()
It has the following arguments:

loss: This value will determine the type of loss function to use in your code. Here i have categorical data in 5 categories or classes so i have used ‘categorical_crossentropy’ loss.
optimizer: This value will determine the type of optimizer function to use in your code.Here i have used Adam optimizer with learning rate 0.001 as it is the best optimizer for categorical data.
metrics: The metrics argument should be a list — you model can have any number of metrics.It is the list of metrics to be evaluated by the model during training and testing.Here i have used accuracy as metric which will compile mu model according to the accuracy.

model.fit_generator()
Fits the model on data yielded batch-by-batch by a Python generator.

It has the following arguments:

generator: The train_generator object that we created earlier.
steps_per_epochs: The steps to take on the training data in one epoch.
epochs: The total number of epochs (pass though the whole dataset once).
callbacks: The list containing all the callbacks that we created earlier.
validation_data: The validation_generator object that we created earlier.
validation_steps: The steps to take on the validation data in one epoch.



In [11]:
model.compile(loss='categorical_crossentropy',
 optimizer = Adam(lr=0.001),
 metrics=['accuracy'])
nb_train_samples = 24176
nb_test_samples = 3006
epochs=25
history=model.fit_generator(
 train_generator,
 steps_per_epoch=nb_train_samples//batch_size,
 epochs=epochs,
 callbacks=callbacks,
 validation_data=test_generator,
 validation_steps=nb_test_samples//batch_size)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-28 12:07:57.982132: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
755/755 [==============================] - 238s 312ms/step - loss: 2.2179 - accuracy: 0.1898 - val_loss: 1.7830 - val_accuracy: 0.2480

Epoch 00001: val_loss improved from inf to 1.78299, saving model to EmotionDetectionModel.h5
Epoch 2/25
755/755 [==============================] - 226s 299ms/step - loss: 1.8416 - accuracy: 0.2296 - val_loss: 1.7873 - val_accuracy: 0.2413

Epoch 00002: val_loss did not improve from 1.78299
Epoch 3/25
755/755 [==============================] - 222s 294ms/step - loss: 1.7994 - accuracy: 0.2473 - val_loss: 1.7489 - val_accuracy: 0.2802

Epoch 00003: val_loss improved from 1.78299 to 1.74894, saving model to EmotionDetectionModel.h5
Epoch 4/25
755/755 [==============================] - 223s 296ms/step - loss: 1.7898 - accuracy: 0.2533 - val_loss: 1.7410 - val_accuracy: 0.2913

Epoch 00004: val_loss improved from 1.74894 to 1.74099, saving model to EmotionDetectionModel.h5
Epoch 5/25
755/755 [==============================] - 224s 296ms/step - lo